<a href="https://colab.research.google.com/github/michael-borck/Foundations-of-Data-Science-with-Python/blob/main/tljh.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# Mount drive
from google.colab import drive
drive.mount('/content/gdrive/')

#Generate root password
import random, string
password = ''.join(random.choice(string.ascii_letters + string.digits) for i in range(20))

#Download ngrok
! wget -q -c -nc https://bin.equinox.io/c/4VmDzA7iaHb/ngrok-stable-linux-amd64.zip
! unzip -qq -n ngrok-stable-linux-amd64.zip

#Setup sshd
! apt-get install -qq -o=Dpkg::Use-Pty=0 openssh-server pwgen > /dev/null

#Set root password
! echo root:$password | chpasswd
! mkdir -p /var/run/sshd
! echo "PermitRootLogin yes" >> /etc/ssh/sshd_config
! echo "PasswordAuthentication yes" >> /etc/ssh/sshd_config
! echo "LD_LIBRARY_PATH=/usr/lib64-nvidia" >> /root/.bashrc
! echo "export LD_LIBRARY_PATH" >> /root/.bashrc

#Run sshd
get_ipython().system_raw('/usr/sbin/sshd -D &')

#Ask token
print("Copy authtoken from https://dashboard.ngrok.com/auth")
import getpass
authtoken = getpass.getpass()

#Create tunnel
get_ipython().system_raw('./ngrok authtoken $authtoken && ./ngrok tcp 22 &')

# Give time for tunnel to be setup otherwise the curl command below fails
import time
time.sleep(2)

# Extract port and host details form tunnel
import os, json
tunnel = os.popen("curl -s http://localhost:4040/api/tunnels")
URL = json.load(tunnel)['tunnels'][0]['public_url'][6:]
HOST, PORT = URL.split(':')

# Details need to ssh from local machine
print("Root password: {}".format(password))
print("To ssh in to this instance run following command on local machine")
print('ssh -p ', PORT, ' root@', HOST, sep='')

In [1]:
# Note working as @ 20 April 2022
# Setup code-server (add missing package needed tljh? From vscode snippet)
!curl -fsSL https://code-server.dev/install.sh | sh

#install python 3.9
!sudo apt-get update -y
!sudo apt-get install python3.9 python3.9-dev

#change alternatives
!sudo update-alternatives --install /usr/bin/python3 python3 /usr/bin/python3.7 1
!sudo update-alternatives --install /usr/bin/python3 python3 /usr/bin/python3.9 2

#check python version
!python --version
#3.9.6

#!sudo apt install python3 python3-dev git curl

!curl -L https://tljh.jupyter.org/bootstrap.py | sudo -E python3 - --admin michael:mest0r

Ubuntu 18.04.5 LTS
Installing v4.3.0 of the amd64 deb package from GitHub.

+ mkdir -p ~/.cache/code-server
+ curl -#fL -o ~/.cache/code-server/code-server_4.3.0_amd64.deb.incomplete -C - https://github.com/coder/code-server/releases/download/v4.3.0/code-server_4.3.0_amd64.deb
######################################################################## 100.0%
+ mv ~/.cache/code-server/code-server_4.3.0_amd64.deb.incomplete ~/.cache/code-server/code-server_4.3.0_amd64.deb
+ dpkg -i ~/.cache/code-server/code-server_4.3.0_amd64.deb
Selecting previously unselected package code-server.
(Reading database ... 155455 files and directories currently installed.)
Preparing to unpack .../code-server_4.3.0_amd64.deb ...
Unpacking code-server (4.3.0) ...
Setting up code-server (4.3.0) ...

deb package has been installed.

To have systemd start code-server now and restart on boot:
  sudo systemctl enable --now code-server@$USER
Or, if you don't want/need a background service you can run:
  code-server
Ig